In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
import xgboost as xgb

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path).fillna("")
dfPred= pd.read_csv(config.original_test_data_path).fillna("")
dfPred2= pd.read_csv(config.original_test_data2_path).fillna("")
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

dfTrain = dfTrain.rename(columns=col_name)
dfPred = dfPred.rename(columns=col_name)
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

dfAll = dfAll.replace({'':np.nan})
dfAll = dfAll.reset_index()

Load data...
Done.


In [3]:
dfAll = pd.read_csv('../../Data/All_Data_dedup.csv')
dfAll.shape

(721, 3456)

In [8]:
raw_predictors = dfAll.columns.tolist()[14:]
raw_predictors

['210X1',
 '210X2',
 '210X3',
 '210X4',
 '210X5',
 '210X6',
 '210X7',
 '210X8',
 '210X9',
 '210X10',
 '210X11',
 '210X12',
 '210X13',
 '210X14',
 '210X15',
 '210X16',
 '210X18',
 '210X19',
 '210X20',
 '210X21',
 '210X23',
 '210X24',
 '210X25',
 '210X26',
 '210X27',
 '210X28',
 '210X29',
 '210X30',
 '210X32',
 '210X33',
 '210X34',
 '210X35',
 '210X36',
 '210X37',
 '210X39',
 '210X40',
 '210X41',
 '210X42',
 '210X43',
 '210X44',
 '210X45',
 '210X46',
 '210X47',
 '210X48',
 '210X49',
 '210X50',
 '210X51',
 '210X52',
 '210X53',
 '210X54',
 '210X55',
 '210X56',
 '210X57',
 '210X58',
 '210X59',
 '210X60',
 '210X61',
 '210X62',
 '210X63',
 '210X64',
 '210X65',
 '210X66',
 '210X67',
 '210X68',
 '210X69',
 '210X70',
 '210X71',
 '210X72',
 '210X73',
 '210X74',
 '210X75',
 '210X76',
 '210X77',
 '210X80',
 '210X81',
 '210X82',
 '210X83',
 '210X84',
 '210X85',
 '210X86',
 '210X87',
 '210X88',
 '210X89',
 '210X90',
 '210X91',
 '210X92',
 '210X93',
 '210X94',
 '210X95',
 '210X96',
 '210X97',
 '210X98

In [9]:
#do the variables minus when two variables' range overlap more than 50%
minus_var = ['ID']
for i in range(len(raw_predictors)-1):
    for j in range(i+1,len(raw_predictors)):
        if (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].max())*(dfAll[raw_predictors[i]].min()-dfAll[raw_predictors[j]].min())<0:
            if float(min(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min()))/max(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min())>=0.5:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)
        elif (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min())*(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())>0:
            if float(min(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())))/max(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min()))>=0.5:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)           

210X1-210X5
210X1-210X39
210X1-210X77
210X1-210X92
210X1-210X93
210X1-210X112
210X1-210X113
210X1-210X118
210X1-210X119
210X1-210X120
210X1-210X121
210X1-210X132
210X1-210X133
210X1-210X134
210X1-210X135
210X1-210X148
210X1-210X149
210X1-210X150
210X1-210X151
210X1-210X153
210X1-210X156
210X1-210X157
210X1-210X162
210X1-210X163
210X1-210X166
210X1-210X167
210X1-210X168
210X1-210X169
210X1-210X170
210X1-210X171
210X1-210X190
210X1-210X218
210X1-220X348
210X1-220X458
210X1-220X467
210X1-220X541
210X1-311X1
210X1-311X76
210X1-311X77
210X1-311X80
210X1-311X82
210X1-311X83
210X1-311X85
210X1-311X86
210X1-311X87
210X1-311X90
210X1-311X91
210X1-311X164
210X1-311X165
210X1-311X168
210X1-311X170
210X1-311X171
210X1-311X173
210X1-311X174
210X1-311X175
210X1-311X178
210X1-311X179
210X1-261X351
210X1-261X354
210X1-261X356
210X1-261X357
210X1-261X359
210X1-261X360
210X1-261X361
210X1-261X364
210X1-261X365
210X1-261X536
210X1-261X537
210X1-261X540
210X1-261X542
210X1-261X543
210X1-261X545
210X1-261X

210X10-220X419
210X10-220X431
210X10-220X471
210X10-220X476
210X10-220X480
210X10-310X3
210X10-310X97
210X10-310X101
210X10-310X105
210X10-310X109
210X10-310X113
210X10-310X117
210X10-310X119
210X10-310X121
210X10-312X97
210X10-312X111
210X10-312X117
210X10-312X123
210X10-312X124
210X10-312X487
210X10-312X488
210X10-312X489
210X10-312X699
210X10-330X35
210X10-330X38
210X10-330X85
210X10-330X88
210X10-330X719
210X10-330X749
210X10-344X251
210X10-344X317
210X10-344X320
210X10-344X323
210X10-420X1
210X10-520X140
210X10-520X399
210X10-520X432
210X10-750X1296
210X11-210X39
210X11-210X94
210X11-210X95
210X11-210X108
210X11-210X110
210X11-210X112
210X11-210X113
210X11-210X219
210X11-210X220
210X11-210X222
210X11-220X3
210X11-220X108
210X11-220X112
210X11-220X116
210X11-220X120
210X11-220X124
210X11-220X128
210X11-220X132
210X11-220X403
210X11-220X419
210X11-220X431
210X11-220X458
210X11-310X3
210X11-310X97
210X11-310X101
210X11-310X105
210X11-310X109
210X11-310X113
210X11-310X117
210X11-310X1

210X25-210X202
210X25-210X203
210X25-210X211
210X25-210X212
210X25-210X226
210X25-210X229
210X25-220X8
210X25-220X485
210X25-220X526
210X25-220X535
210X25-220X551
210X25-310X8
210X25-310X9
210X25-310X19
210X25-310X20
210X25-310X38
210X25-261X318
210X25-261X319
210X25-261X322
210X25-261X323
210X25-261X688
210X25-261X694
210X25-261X695
210X25-312X307
210X25-312X308
210X25-312X310
210X25-312X311
210X25-312X421
210X25-312X430
210X25-340X19
210X25-340X97
210X25-340X105
210X25-344X34
210X25-344X39
210X25-344X40
210X25-344X210
210X25-750X712
210X26-210X37
210X26-210X40
210X26-210X41
210X26-210X42
210X26-210X43
210X26-210X44
210X26-210X45
210X26-210X46
210X26-210X47
210X26-210X48
210X26-210X49
210X26-210X50
210X26-210X51
210X26-210X53
210X26-210X54
210X26-210X55
210X26-210X56
210X26-210X61
210X26-210X62
210X26-210X63
210X26-210X64
210X26-210X69
210X26-210X70
210X26-210X71
210X26-210X72
210X26-210X80
210X26-210X89
210X26-210X116
210X26-210X117
210X26-210X122
210X26-210X123
210X26-210X128
210X26

210X40-210X41
210X40-210X42
210X40-210X43
210X40-210X44
210X40-210X45
210X40-210X46
210X40-210X47
210X40-210X48
210X40-210X49
210X40-210X50
210X40-210X51
210X40-210X53
210X40-210X54
210X40-210X55
210X40-210X56
210X40-210X61
210X40-210X63
210X40-210X64
210X40-210X69
210X40-210X70
210X40-210X71
210X40-210X72
210X40-210X80
210X40-210X89
210X40-210X90
210X40-210X116
210X40-210X117
210X40-210X122
210X40-210X123
210X40-210X128
210X40-210X129
210X40-210X130
210X40-210X131
210X40-210X136
210X40-210X137
210X40-210X146
210X40-210X147
210X40-210X154
210X40-210X155
210X40-210X158
210X40-210X159
210X40-210X164
210X40-210X165
210X40-210X174
210X40-210X175
210X40-210X176
210X40-210X177
210X40-210X181
210X40-210X183
210X40-210X191
210X40-210X199
210X40-210X200
210X40-210X201
210X40-210X202
210X40-210X203
210X40-210X211
210X40-210X212
210X40-210X226
210X40-210X229
210X40-220X8
210X40-220X9
210X40-220X164
210X40-220X168
210X40-220X485
210X40-220X496
210X40-220X507
210X40-220X509
210X40-220X535
210X40-22

210X46-310X20
210X46-310X38
210X46-311X114
210X46-261X318
210X46-261X319
210X46-261X322
210X46-261X323
210X46-261X688
210X46-261X689
210X46-261X694
210X46-261X695
210X46-312X307
210X46-312X308
210X46-312X310
210X46-312X311
210X46-312X421
210X46-312X430
210X46-330X52
210X46-330X53
210X46-330X59
210X46-340X19
210X46-340X59
210X46-340X97
210X46-340X105
210X46-344X34
210X46-344X37
210X46-344X39
210X46-344X40
210X46-344X210
210X46-750X712
210X47-210X53
210X47-210X54
210X47-210X55
210X47-210X56
210X47-210X61
210X47-210X62
210X47-210X63
210X47-210X64
210X47-210X80
210X47-210X102
210X47-210X116
210X47-210X117
210X47-210X122
210X47-210X123
210X47-210X128
210X47-210X129
210X47-210X130
210X47-210X131
210X47-210X136
210X47-210X137
210X47-210X154
210X47-210X155
210X47-210X164
210X47-210X165
210X47-210X174
210X47-210X175
210X47-210X191
210X47-210X199
210X47-210X200
210X47-210X201
210X47-210X202
210X47-210X203
210X47-210X211
210X47-210X212
210X47-210X226
210X47-210X229
210X47-220X8
210X47-220X485
210

...

440AX199-440AX200
440AX199-440AX204
440AX199-440AX205
440AX199-440AX206
440AX200-440AX201
440AX200-440AX202
440AX200-440AX203
440AX200-440AX204
440AX200-440AX205
440AX200-440AX206
440AX201-440AX202
440AX201-440AX203
440AX201-440AX204
440AX201-440AX205
440AX201-440AX206
440AX202-440AX203
440AX202-440AX204
440AX202-440AX205
440AX202-440AX206
440AX203-440AX204
440AX203-440AX205
440AX203-440AX206
440AX204-440AX205
440AX204-440AX206
440AX205-440AX206
440AX207-440AX211
440AX207-440AX212
440AX207-750X294
440AX207-750X775
440AX211-440AX212
440AX212-750X294
520X1-520X144
520X1-520X246
520X1-520X345
520X1-520X351
520X1-520X353
520X3-520X377
520X8-750X115
520X8-750X155
520X8-750X368
520X8-750X742
520X11-520X12
520X11-520X13
520X11-520X21
520X11-520X22
520X11-520X23
520X11-520X24
520X11-520X26
520X11-520X27
520X11-520X28
520X11-520X29
520X11-520X32
520X11-520X37
520X11-520X72
520X11-520X82
520X11-520X106
520X11-520X107
520X11-520X108
520X12-520X13
520X12-520X21
520X12-520X22
520X12-520X24
520X12-5

520X244-520X245
520X246-520X345
520X246-520X351
520X246-520X353
520X252-520X253
520X252-520X254
520X252-520X255
520X252-520X256
520X252-520X257
520X253-520X254
520X253-520X255
520X253-520X256
520X253-520X257
520X254-520X255
520X254-520X256
520X254-520X257
520X255-520X256
520X255-520X257
520X256-520X257
520X258-520X259
520X258-520X260
520X258-520X261
520X258-520X262
520X258-520X263
520X258-520X264
520X258-520X265
520X258-520X266
520X258-520X267
520X259-520X260
520X259-520X261
520X259-520X262
520X259-520X263
520X259-520X264
520X259-520X265
520X259-520X266
520X259-520X267
520X260-520X261
520X260-520X262
520X260-520X263
520X260-520X264
520X260-520X265
520X260-520X266
520X260-520X267
520X261-520X262
520X261-520X263
520X261-520X264
520X261-520X265
520X261-520X266
520X261-520X267
520X262-520X263
520X262-520X264
520X262-520X265
520X262-520X266
520X262-520X267
520X263-520X264
520X263-520X265
520X263-520X266
520X263-520X267
520X264-520X265
520X264-520X266
520X264-520X267
520X265-520X266
520X265-

750X132-750X604
750X132-750X612
750X132-750X620
750X132-750X628
750X132-750X636
750X132-750X644
750X132-750X652
750X132-750X668
750X132-750X676
750X140-750X148
750X140-750X156
750X140-750X164
750X140-750X175
750X140-750X186
750X140-750X197
750X140-750X218
750X140-750X229
750X140-750X240
750X140-750X251
750X140-750X262
750X140-750X273
750X140-750X284
750X140-750X295
750X140-750X306
750X140-750X317
750X140-750X328
750X140-750X336
750X140-750X347
750X140-750X358
750X140-750X369
750X140-750X380
750X140-750X391
750X140-750X402
750X140-750X413
750X140-750X424
750X140-750X435
750X140-750X446
750X140-750X454
750X140-750X465
750X140-750X476
750X140-750X487
750X140-750X498
750X140-750X509
750X140-750X520
750X140-750X531
750X140-750X542
750X140-750X553
750X140-750X564
750X140-750X572
750X140-750X583
750X140-750X594
750X140-750X604
750X140-750X612
750X140-750X620
750X140-750X628
750X140-750X636
750X140-750X644
750X140-750X652
750X140-750X660
750X140-750X668
750X140-750X676
750X140-750X1192
750X148

750X251-750X1192
750X261-750X778
750X261-750X1298
750X262-750X273
750X262-750X284
750X262-750X295
750X262-750X317
750X262-750X336
750X262-750X347
750X262-750X358
750X262-750X391
750X262-750X402
750X262-750X413
750X262-750X424
750X262-750X435
750X262-750X446
750X262-750X454
750X262-750X465
750X262-750X476
750X262-750X487
750X262-750X498
750X262-750X509
750X262-750X520
750X262-750X531
750X262-750X542
750X262-750X553
750X262-750X564
750X262-750X572
750X262-750X583
750X262-750X604
750X262-750X612
750X262-750X620
750X262-750X628
750X262-750X636
750X262-750X652
750X262-750X660
750X262-750X668
750X262-750X676
750X262-750X1192
750X273-750X284
750X273-750X295
750X273-750X306
750X273-750X317
750X273-750X336
750X273-750X347
750X273-750X358
750X273-750X391
750X273-750X402
750X273-750X413
750X273-750X424
750X273-750X435
750X273-750X446
750X273-750X454
750X273-750X465
750X273-750X476
750X273-750X487
750X273-750X498
750X273-750X509
750X273-750X520
750X273-750X531
750X273-750X542
750X273-750X553
750X2

750X435-750X446
750X435-750X454
750X435-750X465
750X435-750X476
750X435-750X487
750X435-750X498
750X435-750X509
750X435-750X520
750X435-750X531
750X435-750X542
750X435-750X553
750X435-750X564
750X435-750X572
750X435-750X583
750X435-750X594
750X435-750X604
750X435-750X612
750X435-750X620
750X435-750X628
750X435-750X636
750X435-750X644
750X435-750X652
750X435-750X660
750X435-750X668
750X435-750X676
750X435-750X1192
750X446-750X454
750X446-750X465
750X446-750X476
750X446-750X487
750X446-750X498
750X446-750X509
750X446-750X520
750X446-750X531
750X446-750X542
750X446-750X553
750X446-750X564
750X446-750X572
750X446-750X583
750X446-750X594
750X446-750X604
750X446-750X612
750X446-750X620
750X446-750X628
750X446-750X636
750X446-750X644
750X446-750X660
750X446-750X668
750X446-750X676
750X446-750X1192
750X454-750X465
750X454-750X476
750X454-750X487
750X454-750X498
750X454-750X509
750X454-750X520
750X454-750X531
750X454-750X542
750X454-750X553
750X454-750X564
750X454-750X572
750X454-750X583
750X45

In [11]:
dfAll = dfAll[minus_var+raw_predictors]
dfAll = dfAll.set_index(dfAll['ID'])
dfTrain_tmp = dfAll.loc[dfTrain['ID']]
dfTrain = pd.merge(dfTrain_tmp,dfTrain[['ID','Y']],'inner','ID')
dfPred = dfAll.loc[dfPred['ID']]

In [12]:
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

In [16]:
f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

predictors = dfTrain.columns.tolist()[1:]+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        try:
            predictors.remove(var)
        except:
            continue
    for var in var_change[key]['category']:
        try:
            predictors.remove(var)
        except:
            continue
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [20]:
from sklearn import metrics
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':3, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,
                         'colsample_bytree':0.4}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =100)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [21]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

[0]	train-rmse:2.33195	eval-rmse:2.32933
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:0.868308	eval-rmse:0.871521
[200]	train-rmse:0.330334	eval-rmse:0.341133
[300]	train-rmse:0.132699	eval-rmse:0.154393
[400]	train-rmse:0.057996	eval-rmse:0.088813
[500]	train-rmse:0.030725	eval-rmse:0.068917
[600]	train-rmse:0.01864	eval-rmse:0.060026
[700]	train-rmse:0.013232	eval-rmse:0.056453
[800]	train-rmse:0.009742	eval-rmse:0.053953
[900]	train-rmse:0.007476	eval-rmse:0.052591
[1000]	train-rmse:0.00584	eval-rmse:0.051727
[1100]	train-rmse:0.004651	eval-rmse:0.051082
[1200]	train-rmse:0.003701	eval-rmse:0.05065
[1300]	train-rmse:0.002992	eval-rmse:0.050379
[1400]	train-rmse:0.002434	eval-rmse:0.050165
[1500]	train-rmse:0.002015	eval-rmse:0.05002
[1600]	train-rmse:0.00166	eval-rmse:0.049907
[1700]	train-rmse:0.00138	eval-rmse:0.049821
[1800]	train-rmse:0.001155	eval-rmse:0.049767
[19

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)
imp.to_csv('../../Submission/imp_%s.csv'%today,header=False,index=False)
test_result.to_csv('../../Submission/test_%s.csv'%today,header=False,index=False)